![](../AnalysisDesigner/datafiles/cms.png)
 
# Start an Analysis in Experimental Particle Physics (3) 

## Part 3: Measure of the Z-boson mass and decay width.



#### Fitting the Z mass peak

Now you have selected in the [Tutorial 2.2](./Part2.ipynb), those pairs of muons compatibles with a Z-boson, you can perform a statistical analysis to measure the Z mass and decay width. 

All code you are going to need is provided in Analyzer_Package, so you can access it as many times as you need. We have implemented two different functions usually used for fitting the Z mass peak: a Gaussian and a Breit-Weigner. 

* [Gaussian](https://en.wikipedia.org/wiki/Normal_distribution) distribution:

$$ G(x;\mu,\sigma) = \dfrac{1}{\sqrt{2\pi}\sigma}\exp[-\dfrac{(x-\mu)^{2}}{2\sigma^{2}}]    $$

* [Relativistic Breit-Wigner](https://en.wikipedia.org/wiki/Relativistic_Breit%E2%80%93Wigner_distribution) distribution:

$$ B(m;M,\Gamma)= N * \dfrac{2}{\pi}*\dfrac{\Gamma^{2}M^{2}}{(m^{2}-M^{2})^{2} + m^{4}(\Gamma^{2}/M^{2})}   $$

To fit a generator-level Z peak a Breit-Wigner fit makes sense. However, reconstructed-level Z peaks have many detector resolutions that smear the Z mass peak. If the detector resolution is relatively poor, then it is usually good enough to fit a gaussian (since the gaussian detector resolution will overwhelm the inherent Briet-Wigner shape of the peak). If the detector resolution is fairly good, then another option is to fit a Breit-Wigner (for the inherent shape) convoluted with a gaussian (to describe the detector effects).This is in the "no-background" case. If you have backgrounds in your sample (Drell-Yan, cosmics, etc...), and you want to do the fit over a large mass range, then another function needs to be included to take care of this - an exponential is commonly used. 
 
**NOTE:** When you are using a python terminal, you need to import ROOT each time you run an exercise 


In [ ]:
# Import ROOT
import ROOT

In [ ]:
cd ~/work/CmsOpenData/AnalysisDesigner

### Gaussian Fit

In [ ]:
# Get the root file that contains the histograms for selected muons: goodHistos.root.
Gfile = ROOT.TFile("datafiles/goodhistos.root", "read")

### Then create again a new the canvas where the histograms are going to be drawn
canvas = ROOT.TCanvas("myCanvas","All muons: Pt",800,600)

histo6=Gfile.Get('h_mass')

histo6.GetXaxis().SetRangeUser(60, 120)


### One more time, draw the histogram
histo6.SetTitle("Invariant Mass")

f=ROOT.TF1("gauss","gaus",60,120)

histo6.Fit("gauss") #use the standard gauss function. 

print("Chi/NDF = {0}".format(f.GetChisquare() / f.GetNDF()))
print("Fit Probability = {0}".format(f.GetProb()))

#self.fit1 = self.gHisto.GetFunction("gaus")
from ROOT import gStyle
gStyle.SetOptFit(11111111)
histo6.Draw()
canvas.Draw()

name = "firstTry"
canvas.SaveAs("../output_histograms/FitGauss"+ name +".png")

### Breit Wigner

In [ ]:
# Breit-Wigner function
def mybw(x, par):
    arg1 = 14.0/22.0 # 2 over pi
    arg2 = par[1]*par[1]*par[2]*par[2] #Gamma=par[1]  M=par[2] 
    arg3 = ((x[0]*x[0]) - (par[2]*par[2]))*((x[0]*x[0]) - (par[2]*par[2]))
    arg4 = x[0]*x[0]*x[0]*x[0]*((par[1]*par[1])/(par[2]*par[2]))
    return par[0]*arg1*arg2/(arg3 + arg4)

In [ ]:
import math

gHisto=Gfile.Get('h_mass')

### Then create again a new the canvas where the histograms are going to be drawn
canvas = ROOT.TCanvas("myCanvas","All muons: Pt",800,600)
gHisto.Draw()
gHisto.GetXaxis().SetRangeUser(60, 120)
gHisto.GetYaxis().SetRangeUser(0, 80)

division = gHisto.GetNbinsX()
massMIN = gHisto.GetBinLowEdge(1)
massMAX = gHisto.GetBinLowEdge(division+1)
BIN_SIZE = gHisto.GetBinWidth(1)

from ROOT import gStyle, TF1
# Create a TF1 object for calling function mybw 
func = TF1("mybw",mybw,massMIN, massMAX,3)

# Set parameter start values for the function
func.SetParameter(0, 1)
func.SetParameter(2, 5)
func.SetParameter(1, 95)

gHisto.Fit("mybw","QR")

gStyle.SetOptFit()
func.Draw("same")

canvas.Draw()


## Printout fit results

chi2 = func.GetChisquare()
chi2_NDF = func.GetChisquare() / func.GetNDF()
prob = func.GetProb()
p1 = func.GetParameter(1)
e1 = func.GetParError(1)
p2 = func.GetParameter(2)
e2 = func.GetParError(2)

print(" ")
print(" ------ Results from Breit-Wigner Fit ------ ")
print(" ")
print(" Chi2 / NDF:      {0}".format(chi2_NDF))
print(" Fit Probability: {0}".format(prob))
print(" Mean:            {0} +- {1}".format(p2,e2))
print(" Gamma:           {0} +- {1}".format(p1,e1/(2*p1))) 


### Breit-Wigner convoluted with a Gaussian

In [ ]:
from ROOT import gROOT, TCanvas, TF1, TF1Convolution

myNewHisto=Gfile.Get('h_mass')

mathVoigt = TF1("mathVoigt","[3]*TMath::Voigt(x-[0],[1],[2],4)",0,200)

mathVoigt.SetParameter(0,myNewHisto.GetMean())
mathVoigt.SetParameter(1,myNewHisto.GetRMS())
mathVoigt.SetParameter(2,2.4)
mathVoigt.SetParameter(3,400)

mathVoigt.SetParName(0,"Z mass")
mathVoigt.SetParName(1,"Experimental Resolution")
mathVoigt.SetParName(2,"Z Width")
mathVoigt.SetParName(3,"Overall Normalization")

canvas3 = ROOT.TCanvas("Voigtian","Voigtian",800,600)

myNewHisto.SetLineColor(4)
myNewHisto.Draw()
myNewHisto.Fit("mathVoigt")

canvas3.Draw()

print("Fit Output:")
print("Z Boson Mass            = ", mathVoigt.GetParameter(0))
print("Z Boson Width           = ", mathVoigt.GetParameter(2))
print("Experimental Resolution = ", mathVoigt.GetParameter(1))
print("Overall Normalization   = ", mathVoigt.GetParameter(3))
print("")
print("Normalized Chi2         = ", mathVoigt.GetChisquare() / mathVoigt.GetNDF())
print("Fit Probability         = ", mathVoigt.GetProb())